# Periodic (Welch's method) and aperiodoc (FOOOF) activity estimations
## Importing needed packages

In [ ]:
import csv
import matplotlib
import pathlib
import os
import numpy as np
import pandas as pd
# no need to use np. anymore
from pylab import *
# MNE-Python tools
import mne
from mne import read_source_estimate
# for Welch's method
from scipy import signal
# Import the FOOOFGroup object
from fooof import FOOOFGroup

## Extracting the data

In [ ]:
csvreader = pd.read_csv('/archive/21098_pd_dimensionality/metadata/sub_group.csv', sep = ';')    
csvreader = array(csvreader)

sub_type = zeros(csvreader.shape[0],dtype=int)
for k in range(0,csvreader.shape[0]):
    if 'control' in csvreader[k,1]:
        sub_type[k] = 1
        
# Paths to data
MEG_path = os.path.join('/archive/21098_pd_dimensionality/MEG')
data_paths = os.listdir(MEG_path)
file_collection = [os.listdir(f'{MEG_path}/{p}') for p in data_paths]

label_ses1_final_paths = []
label_ses2_final_paths = []
for data_path, files in zip(data_paths, file_collection):
    label_ses1_final_paths += [f'{MEG_path}/{data_path}/{file}' for file in files if 'ses1-labeltc.npy' in file]
    label_ses2_final_paths += [f'{MEG_path}/{data_path}/{file}' for file in files if 'ses2-labeltc.npy' in file]

# Extracting the data
label_ses1_data = [np.load(label_fname) for label_fname in label_ses1_final_paths]
label_ses2_data = [np.load(label_fname) for label_fname in label_ses2_final_paths]

data_all = [ [] for i in range(4) ]
for i in range(0,n_indiv):
    if sub_type[i] == 0 :
        data_all[0].append(label_ses1_data[i])
        data_all[1].append(label_ses2_data[i])
    else :
        data_all[2].append(label_ses1_data[i])
        data_all[3].append(label_ses2_data[i])

## Estimating Welch's PSD and the peak frequencies

In [ ]:
# Sampling frequency of the time series
fs = 200
# Desired window to use
window = 'hann'
# Length of each segment
nperseg = 1000
# Number of points to overlap between segments
noverlap = nperseg * 0.5

Welch_all = [ [] for i in range(4) ]

for i in range(0,4):
    for j in range(0,len(data_all[i])):
        x = data_all[i][j]
        f_welch, Welch_xx = signal.welch(x, fs=fs, nperseg=nperseg, noverlap=noverlap)
        Welch_all[i].append(Welch_xx)

fg_all = [ [] for i in range(4) ]

max_n_peaks = 4
min_peak_height = 0.2
peak_width_limits = [1,10]

# Define frequency range across which to model the spectrum
f_lo = 1
f_hi = 45
freq_range = [f_lo, f_hi]

for i in range(0,4):
    for j in range(0,len(Welch_all[i])):
        # Initialize a FOOOFGroup object, which accepts all the same settings as FOOOF
        fg = FOOOFGroup(peak_width_limits = peak_width_limits, min_peak_height = min_peak_height, max_n_peaks=max_n_peaks)
        # Fit a group of power spectra with the .fit() method
        fg.fit(f_welch, Welch_all[i][j], freq_range)
        fg_all[i].append(fg)

## Estimating Welch's PSD and 1/f-exponents temporal dynamics

In [ ]:
# Sampling frequency of the time series
fs = 200
# Length of each segment in [ms]
nperseg = 400
# Number of points to overlap between segments
noverlap = nperseg * 0.8
# Length of one segment = 50sec
n_seg = 25*nperseg
# Increment = 10sec
increment = int(n_seg*0.2)

# Initialising the Welch's PSD list
Welch_time_all = [ [] for i in range(4) ]
for i in range(0,n_indiv):
    if sub_type[i] == 0 :
        Welch_time_all[0].append([])
        Welch_time_all[1].append([])
    else :
        Welch_time_all[2].append([])
        Welch_time_all[3].append([])

for i in range(0,4):
    for j in range(0,len(data_all[i])):
        for k in range(0,data_all[i][j].shape[0]):
            x_src = data_all[i][j][k,:]
            n_times = len(x_src)
            # First segment
            x_src_seg = x_src[0:n_seg]
            Welch_xx = []
            f_welch_time, Welch_xx = signal.welch(x_src_seg, fs=fs, nperseg=nperseg, noverlap=noverlap)
            i0 = increment
            while i0 + n_seg < n_times:
                x_src_seg = x_src[i0:i0+n_seg]
                f_welch_time, welch_ = signal.welch(x_src_seg, fs=fs, nperseg=nperseg, noverlap=noverlap)
                Welch_xx = np.c_[ Welch_xx, welch_ ]
                i0 += increment
            Welch_time_all[i][j].append(Welch_xx)


f = arange(0.,101)
fg_time_all = [ [ [] for j in range(0,len(Welch_time_all[i]))] for i in range(0,4)]
# Define the fooof parameters
peak_width_limits = [2, 10]
min_peak_height = 0.2
max_n_peaks = 4
# Define frequency range across which to model the spectrum
f_lo = 1
f_hi = 45
freq_range = [f_lo, f_hi]
for i in range(0,4):
    for j in range(0,len(Welch_time_all[i])):
        for k in range(0,len(Welch_time_all[i][j])):
            # Initialize a FOOOFGroup object, which accepts all the same settings as FOOOF
            fg = FOOOFGroup(peak_width_limits = peak_width_limits,
                              min_peak_height = min_peak_height,
                                  max_n_peaks = max_n_peaks)
            # Fit a group of power spectra with the .fit() method
            fg.fit(f, Welch_time_all[i][j][k].T, freq_range)
            fg_time_all[i][j].append(fg)